# Intensity Normalization
This script normalizes runs global intensity normalization with grand mean = 10000 across for group analysis.

-----------------------------------------------------------
Script written by Sepideh Tabrik & Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
import glob
import os
import subprocess
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find smoothed BOLD files
bold_dirs = list(data_path.glob("sub*/ses-0*/func/rest*/*smoothed.nii.gz"))

# Process each BOLD file
for curr_bold in bold_dirs:
    sub_dir = curr_bold.parent  # Use pathlib to get the parent directory
    
    print(f"Current file is: {curr_bold}")

    # Change directory only if it exists
    if sub_dir.exists():
        print(f"\t1) Changing the working directory to {sub_dir}")
        os.chdir(sub_dir)
    else:
        print(f"\tWarning: {sub_dir} does not exist, skipping.")
        continue

    # Calculate median intensity
    print("\t2) Calculating median intensity...")
    try:
        median_intensity = float(subprocess.check_output(f"fslstats {curr_bold.name} -k mean_func_brain_mask.nii.gz -p 50", shell=True).decode().strip())
        print(f"\t\tMedian intensity is {median_intensity}")
    except subprocess.CalledProcessError:
        print(f"\tWarning: Failed to calculate median intensity for {curr_bold}, skipping.")
        continue

    # Intensity Normalization with grand mean = 10000
    print("\t3) Performing intensity normalization...")
    normmean = 10000

    if median_intensity > 0:  # Avoid division by zero
        scaling = normmean / median_intensity
        output_file = sub_dir / f"{curr_bold.stem.split('.')[0]}_intnorm.nii.gz"
        os.system(f"fslmaths {curr_bold.name} -mul {scaling} {output_file}")
        print(f"\t\tSaved normalized file: {output_file}")
    else:
        print(f"\tWarning: Median intensity is zero, skipping normalization.")

    print()
